# Study the impact of the news

In [2]:
import os
import sys
import re
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
from pyhanlp import *
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import matplotlib.pyplot as plt
from gensim import corpora
from collections import defaultdict
from snownlp import SnowNLP
from updatedb import UpdateDB
from copy import deepcopy

HanLP.Config.ShowTermNature = False

%matplotlib inline

### 1. 处理价格数据

In [2]:
updb = UpdateDB('10.24.224.249','fineng','123456')
index = updb.pull_index(['000300.SH']).loc['20100101':].reset_index().rename(columns={'TRADE_DT':'date'}).sort_values('date')
index['next_date'] = index['date'].shift(-1)
index.head()

S_INFO_WINDCODE,date,000300.SH,next_date
0,20100104,3535.229,20100105
1,20100105,3564.038,20100106
2,20100106,3541.727,20100107
3,20100107,3471.456,20100108
4,20100108,3480.130,20100111


In [3]:
code_name = updb.pull_data_accor_date('MyAShareDescription','wind',set_index=False).set_index('S_INFO_WINDCODE')
code_name

,S_INFO_NAME
S_INFO_WINDCODE,
000001.SZ,平安银行
000002.SZ,万科A
000003.SZ,PT金田A(退市)
000004.SZ,国农科技
000005.SZ,世纪星源
000006.SZ,深振业A
000007.SZ,全新好
000008.SZ,神州高铁
000009.SZ,中国宝安


## 2. 处理新闻数据

In [3]:
# newsfiles = os.listdir('../news2/000001.SZ')
# newsfiles = list(filter(lambda x: x.split('.')[1][0]=='S', newsfiles))
symbol = '000001.SZ'
files = os.listdir('../news2/%s/' % symbol)
news = pd.concat([pd.read_csv('../news2/%s/%s' % (symbol,f), dtype={'date':str}) for f in files])
# files
# news = pd.read_csv('../news2/%s/%s' % (symbol, files[0]),dtype={'date':str})[['date','title','time']]
news

,comment,content,date,read,time,title,url
0,0,"平安银行(000001)2020-03-05融资融券信息显示，平安银行融资余额3,562...",2020-03-06,369,09:09:55,平安银行(000001)融资融券信息(03-05),"http://guba.eastmoney.com/news,000001,91004066..."
1,0,平安银行融资融券信息显示，2020年3月2日融资净偿还1426.36万元；融资余额33.99...,2020-03-03,579,08:47:03,平安银行：融资净偿还1426.36万元，融资余额33.99亿元（03-02）,"http://guba.eastmoney.com/news,000001,90883759..."
2,0,"平安银行(000001)2020-03-02融资融券信息显示，平安银行融资余额3,399...",2020-03-03,637,09:12:14,平安银行(000001)融资融券信息(03-02),"http://guba.eastmoney.com/news,000001,90884279..."
3,0,平安银行融资融券信息显示，2020年3月3日融资净偿还844.89万元；融资余额33.91亿...,2020-03-04,643,08:45:31,平安银行：融资净偿还844.89万元，融资余额33.91亿元（03-03）,"http://guba.eastmoney.com/news,000001,90921846..."
4,0,"平安银行(000001)2020-03-03融资融券信息显示，平安银行融资余额3,390...",2020-03-04,648,09:10:44,平安银行(000001)融资融券信息(03-03),"http://guba.eastmoney.com/news,000001,90922359..."
5,0,平安银行融资融券信息显示，2020年3月4日融资净买入2614.87万元；融资余额34.17...,2020-03-05,593,08:42:37,平安银行：融资净买入2614.87万元，融资余额34.17亿元（03-04）,"http://guba.eastmoney.com/news,000001,90960205..."
6,0,平安银行融资融券信息显示，2020年2月11日融资净偿还510.8万元；融资余额32.6亿元...,2020-02-12,757,08:45:25,平安银行：融资净偿还510.8万元，融资余额32.6亿元（02-11）,"http://guba.eastmoney.com/news,000001,90305620..."
7,1,平安银行今日发布上半年业绩报告。该行上半年实现净利润133.72亿元，同比增长6.5%...,2018-08-15,11517,17:30:07,平安银行上半年净利润133.72亿元 零售贡献占67.9%,"http://guba.eastmoney.com/news,000001,77849654..."
8,1,"平安银行(000001)2018-10-15融资融券信息显示，平安银行融资余额4,176,0...",2018-10-16,7220,08:47:41,平安银行(000001)融资融券信息(10-15),"http://guba.eastmoney.com/news,000001,78707774..."
9,0,"平安银行(000001)2018-12-04融资融券信息显示，平安银行融资余额4,136,6...",2018-12-05,4308,09:34:23,平安银行(000001)融资融券信息(12-04),"http://guba.eastmoney.com/news,000001,79606356..."


In [9]:
','.join(news['url'].values[0].split(',')[-2:])

'000001,910040666.html'

In [6]:
newsfiles = os.listdir('../news')
newsfiles = list(filter(lambda x: x.split('.')[1][0]=='S', newsfiles))
newsfiles

['002551.SZ.csv',
 '000935.SZ.csv',
 '603798.SH.csv',
 '600518.SH.csv',
 '300515.SZ.csv',
 '300274.SZ.csv',
 '603567.SH.csv',
 '002030.SZ.csv',
 '603800.SH.csv',
 '300038.SZ.csv',
 '000597.SZ.csv',
 '300167.SZ.csv',
 '601811.SH.csv',
 '600507.SH.csv',
 '603638.SH.csv',
 '603181.SH.csv',
 '300476.SZ.csv',
 '300341.SZ.csv',
 '002438.SZ.csv',
 '300470.SZ.csv',
 '300517.SZ.csv',
 '300406.SZ.csv',
 '603309.SH.csv',
 '603660.SH.csv',
 '300044.SZ.csv',
 '002309.SZ.csv',
 '300622.SZ.csv',
 '300391.SZ.csv',
 '300053.SZ.csv',
 '300117.SZ.csv',
 '600038.SH.csv',
 '600589.SH.csv',
 '601388.SH.csv',
 '600653.SH.csv',
 '600526.SH.csv',
 '300711.SZ.csv',
 '000858.SZ.csv',
 '002387.SZ.csv',
 '300737.SZ.csv',
 '300007.SZ.csv',
 '600289.SH.csv',
 '601216.SH.csv',
 '300400.SZ.csv',
 '600774.SH.csv',
 '603908.SH.csv',
 '000987.SZ.csv',
 '300636.SZ.csv',
 '603650.SH.csv',
 '600798.SH.csv',
 '603321.SH.csv',
 '300556.SZ.csv',
 '000657.SZ.csv',
 '600648.SH.csv',
 '600012.SH.csv',
 '600877.SH.csv',
 '000823.S

### 3. 停词表中加入公司名

In [27]:
def get_price(symbol):
    price = updb.pull_stocks([symbol], startT='20100101').sort_index().reset_index().rename(columns={'TRADE_DT':'date',symbol:'close'})
    price['CloseRet_t-2_t+1'] = price['close'].shift(-1) / price['close'].shift(2)-1
    price['CloseRet_t-1_t+1'] = price['close'].shift(-1) / price['close'].shift(1)-1
    price['CloseRet_t_t+1'] = price['close'].shift(-1) / price['close'].shift(0)-1
    price['CloseRet_t-1_t'] = price['close'].shift(0) / price['close'].shift(1)-1
    return price

def get_label(df, indicators, thresholds):
    def assign(df, indicator,threshold):
        pos = df.loc[df[indicator]>threshold,['title','date']]
        pos['label'] = 1
        neg = df.loc[df[indicator]<-threshold,['title','date']]
        neg['label'] = 0
        df_ok = pd.concat([pos,neg])
        df_ok['title'] = [s.replace(code_name.loc[symbol].values[0],'') for s in df_ok['title']]
        return df_ok
    dfs = [assign(deepcopy(df),ind, thr) for ind,thr in zip(indicators,thresholds)]
    return dfs
    
def process(symbol, indicators, thresholds, sep_date, first=False):
    # process news
    news = pd.read_csv('../news/%s.csv' % symbol,dtype={'date':str})[['date','title','time']]
    news = news.dropna()
    news = news.sort_values('date')
    nonsense=['融资融券信息','大宗交易','今日超大单流','龙虎榜','下跌','上涨','跌幅','涨幅','涨停','跌停',
              '大涨','大跌','跳水','盘中','融资余额','反弹','回调','火箭发射']
    is_useful = np.array([not any(bool(re.search(w,l)) for w in nonsense) for l in news['title'].values])
    news = news[is_useful]
    # news['content'] = news['title'] + news['content']
    news['date'] = list(map(lambda x:x.replace('-',''), news['date'].values))

    news = news.merge(index[['date','next_date']],how='left')
    news.loc[news['time']>'13:00:00','date'] = np.nan
    news['date'] = news['date'].fillna(news['next_date'])

    price = get_price(symbol)
    news = news.merge(price)
    dfs = get_label(news, indicators, thresholds)
    
    for ind,df in zip(indicators, dfs):
        df_use = df.loc[df['date']<sep_date, ['label','title']]
        if first:
            df_use.to_csv('output/%s.csv' % ind, index=False)
        else:
            df_use.loc[df['date']<sep_date, ['label','title']].to_csv('output/%s.csv' % ind, mode='a', header=False,  index=False)
#         print('useful: %d  tot:%d' % (len(df_use),len(df)),)
#     print()
    
indicators = ['CloseRet_t-2_t+1','CloseRet_t-1_t+1','CloseRet_t_t+1']
thresholds = [0.1,0.08,0.05]
sep_date='20180101'
nf = newsfiles[0]
process(nf[:9],indicators, thresholds, sep_date, first=True)
for i,nf in enumerate(newsfiles[1:5]):
    if (len(nf)==13):
        print('processing %s %d/%d' % (nf[:9], i, len(newsfiles)))
        process(nf[:9],indicators, thresholds, sep_date, first=False)


processing 000935.SZ 0/3668
processing 603798.SH 1/3668
processing 600518.SH 2/3668
processing 300515.SZ 3/3668


In [5]:
import pandas as pd
from updatedb import UpdateDB

In [14]:
updb = UpdateDB('10.24.224.249','fineng','123456')
code_name = updb.pull_data_accor_date('MyAShareDescription','wind',set_index=False).set_index('S_INFO_WINDCODE')
code_name

,S_INFO_NAME,S_INFO_LISTDATE,S_INFO_DELISTDATE
S_INFO_WINDCODE,,,
300407.SZ,凯发电气,20141203,None
603901.SH,永创智能,20150529,None
300374.SZ,恒通科技,20150319,None
002747.SZ,埃斯顿,20150320,None
002780.SZ,三夫户外,20151209,None
300429.SZ,强力新材,20150324,None
300440.SZ,运达科技,20150423,None
002729.SZ,好利来,20140912,None
603918.SH,金桥信息,20150528,None


In [17]:
df = pd.read_csv('/home/wy/Downloads/EastMoney.csv')
df['label'] = 0
df = df.rename(columns={'TITLE':'x_test','S_INFO_WINDCODE':'code'})
df = df[df['code']!='000043.SZ']
df['x_test'] = [s.replace(code_name.loc[c].values[0],'') for c,s in zip(df['code'], df['x_test'])]
df


,code,URL,x_test,label
0,600102.SH,"news,600102,45663295",莱钢股份和济南钢铁换股过会 明起双双复牌,0
1,600102.SH,"news,600102,47792148",莱钢股份6.88亿限售股18日解禁上市,0
2,600102.SH,"news,600102,48739078",莱钢股份28日起终止上市,0
3,600102.SH,"news,600102,48740016",莱钢股份资产重组实施 28日起终止上市,0
4,600102.SH,"news,600102,48767335",莱钢股份28日退市 山东钢铁即将登场,0
5,600102.SH,"news,600102,48878709",莱钢股份今日告别A股 山钢整合下一步欲吞日钢,0
6,600102.SH,"news,600102,60830770",莱钢股份整体上市启动 后市波段操作,0
7,601799.SH,"news,601799,40807215",11日起停牌 筹划股权激励计划事宜,0
8,601799.SH,"news,601799,40977197",远低于市价推股票激励,0
9,601799.SH,"news,601799,40985532",股权激励授予价远低市价,0


In [18]:
df[['label','x_test']].to_csv('/home/wy/Downloads/test22.csv',sep='\t',index=False)
df[['URL']].to_csv('/home/wy/Downloads/URL.csv')

In [23]:
score = pd.read_csv('/home/wy/Downloads/test_results.tsv',sep='\t',header=None)

,0,1
0,0.553037,0.446963
1,0.687471,0.312529
2,0.613366,0.386634
3,0.613298,0.386702
4,0.861878,0.138122
5,0.608871,0.391129
6,0.087007,0.912993
7,0.153081,0.846919
8,0.267346,0.732654
9,0.151976,0.848024


In [26]:
from pymysql import connect

In [28]:
df_results = pd.DataFrame({'SCORE': score.iloc[:,1].values, 'URL': df['URL']})
df_results
conn = connect(host='10.24.224.249', port=3306, database='webdata', user='wy', password=',.,.,l',
               charset='utf8')
cur = conn.cursor()
cnt = 0
for url, score in zip(df_results['URL'], df_results['SCORE']):
    sql = "update EastMoney set SCORE=%f where URL=\'%s\'" % (score, url)
    cur.execute(sql)
    cnt+=1
    if cnt % 1000 ==0:
        print('finish %d ' % cnt)
cur.close()
conn.close()

finish 1000 
finish 2000 
finish 3000 
finish 4000 
finish 5000 
finish 6000 
finish 7000 
finish 8000 
finish 9000 
finish 10000 
finish 11000 
finish 12000 
finish 13000 
finish 14000 
finish 15000 
finish 16000 
finish 17000 
finish 18000 
finish 19000 
finish 20000 
finish 21000 
finish 22000 
finish 23000 
finish 24000 
finish 25000 
finish 26000 
finish 27000 
finish 28000 
finish 29000 
finish 30000 
finish 31000 
finish 32000 
finish 33000 
finish 34000 
finish 35000 
finish 36000 
finish 37000 
finish 38000 
finish 39000 
finish 40000 
finish 41000 
finish 42000 
finish 43000 
finish 44000 
finish 45000 
finish 46000 
finish 47000 
finish 48000 
finish 49000 
finish 50000 
finish 51000 
finish 52000 
finish 53000 
finish 54000 
finish 55000 
finish 56000 
finish 57000 
finish 58000 
finish 59000 
finish 60000 
finish 61000 
finish 62000 
finish 63000 
finish 64000 
finish 65000 
finish 66000 
finish 67000 
finish 68000 
finish 69000 
finish 70000 
finish 71000 
finish 72000 
f

finish 555000 
finish 556000 
finish 557000 
finish 558000 
finish 559000 
finish 560000 
finish 561000 
finish 562000 
finish 563000 
finish 564000 
finish 565000 
finish 566000 
finish 567000 
finish 568000 
finish 569000 
finish 570000 
finish 571000 
finish 572000 
finish 573000 
finish 574000 
finish 575000 
finish 576000 
finish 577000 
finish 578000 
finish 579000 
finish 580000 
finish 581000 
finish 582000 
finish 583000 
finish 584000 
finish 585000 
finish 586000 
finish 587000 
finish 588000 
finish 589000 
finish 590000 
finish 591000 
finish 592000 
finish 593000 
finish 594000 
finish 595000 
finish 596000 
finish 597000 
finish 598000 
finish 599000 
finish 600000 
finish 601000 
finish 602000 
finish 603000 
finish 604000 
finish 605000 
finish 606000 
finish 607000 
finish 608000 
finish 609000 
finish 610000 
finish 611000 
finish 612000 
finish 613000 
finish 614000 
finish 615000 
finish 616000 
finish 617000 
finish 618000 
finish 619000 
finish 620000 
finish 621